In [1]:
import numba as nb
import numpy as np

In [2]:
@nb.jitclass((("size", nb.int32), ("array", nb.float64[:])))
class ArrayBuilder:
    def __init__(self, initial_capacity):
        self.size = 0
        self.array = np.empty(initial_capacity, dtype=nb.float64)

    def append(self, value):
        if self.size == len(self.array):
            n = self.size // 2 + 1
            self.array = np.append(self.array, np.empty(n, dtype=self.array.dtype))
        self.array[self.size] = value
        self.size += 1

    def get(self):
        return self.array[: self.size]

In [43]:
x = np.random.randn(100000)

In [44]:
ar = ArrayBuilder(0)

@nb.njit
def fill(x):
    ar= ArrayBuilder(0)
    for xi in x:
        ar.append(xi)
    return ar.get()
fill(x)

%timeit fill(x)

3.08 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
@nb.njit
def fill(x):
    a = np.empty(0)
    n = 0
    for xi in x:
        if n == len(a):
            m = len(a) // 2 + 1
            a = np.append(a, np.empty(m))
        a[n] = xi
        n+=1
    return a[:n]
fill(x)

%timeit fill(x)

199 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
@nb.njit
def grow(a, n):
    if n == len(a):
        m = n // 2 + 1
        a2 = np.empty(n + m)
        a2[:n] = a
        return a2
    return a
        
@nb.njit
def fill(x):
    a = np.empty(0)
    n = 0
    for xi in x:
        a = grow(a, n)
        a[n] = xi
        n+= 1
    return a[:n]
fill(x)

%timeit fill(x)

1.74 ms ± 38.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
@nb.njit
def fill(x):
    a = np.empty(0)
    n = 0
    for xi in x:
        if n == len(a):
            m = n // 2 + 1
            a = np.append(a, np.empty(m))
        a[n] = xi
        n+= 1
    return a[:n]
fill(x)

%timeit fill(x)

23.7 µs ± 751 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
@nb.njit
def fill(x):
    a = np.empty(0)
    n = 0
    for xi in x:
        if n == len(a):
            m = n // 2 + 1
            a2 = np.empty(n + m)
            a2[:n] = a
            a = a2
        a[n] = xi
        n+= 1
    return a[:n]
fill(x)

%timeit fill(x)

21.7 µs ± 740 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
